# Data

Please load the dataset files "train.txt" and "test.txt"

Télécharger le dossier data.zip, dans la section fichiers, clique droit, importer "data.zip"

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
   creating: data/restaurant/
  inflating: __MACOSX/data/._restaurant  
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/laptop/
  inflating: __MACOSX/data/._laptop  
  inflating: data/restaurant/.DS_Store  
  inflating: __MACOSX/data/restaurant/._.DS_Store  
  inflating: data/restaurant/train.txt  
  inflating: __MACOSX/data/restaurant/._train.txt  
  inflating: data/restaurant/test.txt  
  inflating: __MACOSX/data/restaurant/._test.txt  
  inflating: data/laptop/.DS_Store   
  inflating: __MACOSX/data/laptop/._.DS_Store  
  inflating: data/laptop/train.txt   
  inflating: __MACOSX/data/laptop/._train.txt  
  inflating: data/laptop/test.txt    
  inflating: __MACOSX/data/laptop/._test.txt  


Download the 300-dimensional pre-trained word vectors
 
temps : 8 mins pour la cellule ci dessous

In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip -q glove.840B.300d.zip

--2022-01-07 04:45:47--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2022-01-07 04:45:47--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2022-01-07 04:45:47--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [ ]:
!unzip -q glove.840B.300d.zip.1

unzip:  cannot find or open glove.840B.300d.zip.1, glove.840B.300d.zip.1.zip or glove.840B.300d.zip.1.ZIP.


## Tokens


The goal of this section is to tokenize our raw data into tokens. 
We first brek our sentences into words, each word has a unique assigned token 

In [ ]:
import os
import ast
import spacy
import numpy as np
from errno import ENOENT
from collections import Counter

nlp = spacy.load("en")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np

In [ ]:
def get_token(token, words):
  
  words_count = Counter(words).most_common()
  for word, _ in words_count:
    if word not in token and ' ' not in word and '\n' not in word and 'aspect_term' not in word:
      token[word] = len(token)
  
  return token


In [ ]:
def tokenize(dataset, ftrain,ftest ):

  words_token, max_aspect_len, max_context_len = {},0, 0
  words_token['<pad>'] = 0


  filename = dataset + ftrain
  
  lines = open(filename,'r').readlines()
  words = list()

  for i in range (0, len(lines),3):
    context_sptoks = nlp(lines[i].strip())
    words.extend([sp.text.lower() for sp in context_sptoks])
    if len(context_sptoks) - 1 > max_context_len:
      max_context_len = len(context_sptoks) - 1
   
    aspect_sptoks = nlp(lines[i + 1].strip())

    if len(aspect_sptoks) > max_aspect_len:
      max_aspect_len = len(aspect_sptoks)
    words.extend([sp.text.lower() for sp in aspect_sptoks])

  filename = dataset + ftest
  print(filename)
  lines = open(filename,'r').readlines()
  words = list()

  for i in range (0, len(lines),3):
    context_sptoks = nlp(lines[i].strip())
    words.extend([sp.text.lower() for sp in context_sptoks])
    if len(context_sptoks) - 1 > max_context_len:
      max_context_len = len(context_sptoks) - 1
   
    aspect_sptoks = nlp(lines[i + 1].strip())

    if len(aspect_sptoks) > max_aspect_len:
      max_aspect_len = len(aspect_sptoks)
    words.extend([sp.text.lower() for sp in aspect_sptoks])

  words_token = get_token(words_token, words)

  return words_token, max_aspect_len, max_context_len



In [ ]:
def read_from_token(token, max_aspect_len, max_context_len,dataset, path_dataset):
  fname = path_dataset+dataset + '.txt'
  
  aspects, contexts, labels, aspect_lens, context_lens = list(), list(), list(), list(), list()

  if not os.path.isfile(fname):
    raise IOError(ENOENT, 'Not a file', fname)
  
  lines = open(fname, 'r').readlines()

  for i in range(0, len(lines), 3):

    polarity = lines[i + 2].split()[0]
    
    if polarity == 'negative':
      labels.append(0)
    elif polarity == 'neutral':
      labels.append(1)
    elif polarity == 'positive':
      labels.append(2)

    context_sptoks = nlp(lines[i].strip())
    context = []
    for sptok in context_sptoks:
      if sptok.text.lower() in token:
        context.append(token[sptok.text.lower()])
 
    aspect_sptoks = nlp(lines[i + 1].strip())
    aspect = []
    for aspect_sptok in aspect_sptoks:
      if aspect_sptok.text.lower() in token:
        aspect.append(token[aspect_sptok.text.lower()])
    
    
    aspects.append(aspect + [0] * (max_aspect_len - len(aspect)))
    contexts.append(context + [0] * (max_context_len - len(context)))
    aspect_lens.append(len(aspect_sptoks))
    context_lens.append(len(context_sptoks) - 1)
    
  aspects = np.asarray(aspects)
  contexts = np.asarray(contexts)
  labels = np.asarray(labels)
  aspect_lens = np.asarray(aspect_lens)
  context_lens = np.asarray(context_lens)
  
  return(aspects, contexts, labels, aspect_lens, context_lens)
 
  



In [ ]:
def save_file(dataset,aspects, contexts, labels, aspects_lens, contexts_lens,path_dataset):
  save_fname = path_dataset+dataset + '.npz'

  np.savez(save_fname, aspects=aspects, contexts=contexts, labels=labels, aspect_lens=aspects_lens, context_lens=contexts_lens)
  
  return save_fname


## Dataset

In [ ]:
class IanDataset(Dataset):

    def __init__(self, path):
        data = np.load(path)
        self.aspects = torch.from_numpy(data['aspects']).long()
        self.contexts = torch.from_numpy(data['contexts']).long()
        self.labels = torch.from_numpy(data['labels']).long()
        self.aspect_lens = torch.from_numpy(data['aspect_lens']).long()
        self.context_lens = torch.from_numpy(data['context_lens']).long()
        self.len = self.labels.shape[0]
        aspect_max_len = self.aspects.size(1)
        context_max_len = self.contexts.size(1)
        self.aspect_mask = torch.zeros(aspect_max_len, aspect_max_len)
        self.context_mask = torch.zeros(context_max_len, context_max_len)
        for i in range(aspect_max_len):
            self.aspect_mask[i, 0:i + 1] = 1
        for i in range(context_max_len):
            self.context_mask[i, 0:i + 1] = 1

    def __getitem__(self, index):
        return self.aspects[index], self.contexts[index], self.labels[index], \
               self.aspect_mask[self.aspect_lens[index] - 1], self.context_mask[self.context_lens[index] - 1]

    def __len__(self):
        return self.len

In [ ]:
def load_word_embeddings(fname, embedding_dim, word2id):
    if not os.path.isfile(fname):
        raise IOError(ENOENT, 'Not a file', fname)

    word2vec = np.random.uniform(-0.01, 0.01, [len(word2id), embedding_dim])
    oov = len(word2id)
    with open(fname, 'r', encoding='utf-8') as f:
        for line in f:
            content = line.split(' ')
            if content[0] in word2id:
                word2vec[word2id[content[0]]] = np.array(list(map(float, content[1:])))
                oov = oov - 1
    word2vec[word2id['<pad>'], :] = 0
    print('There are %s words in vocabulary and %s words out of vocabulary' % (len(word2id) - oov, oov))
    return word2vec

In [ ]:
def get_word(tokens, tok):
  return(list(tokens.keys())[list(tokens.values()).index(tok)])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np

class Attention(nn.Module):

    def __init__(self, query_size, key_size):
        super(Attention, self).__init__()
        self.weights = nn.Parameter(torch.rand(key_size, query_size) * 0.2 - 0.1)
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, query, key, mask):
        # query: (batch_size, query_size)
        # key: (batch_size, time_step, key_size)
        # value: (batch_size, time_step, value_size)
        # mask: (batch_size, time_step)
        batch_size = key.size(0)
        time_step = key.size(1)
        weights = self.weights.repeat(batch_size, 1, 1) # (batch_size, key_size, query_size)
        query = query.unsqueeze(-1)    # (batch_size, query_size, 1)
        mids = weights.matmul(query)    # (batch_size, key_size, 1)
        mids = mids.repeat(time_step, 1, 1, 1).transpose(0, 1) # (batch_size, time_step, key_size, 1)
        key = key.unsqueeze(-2)    # (batch_size, time_step, 1, key_size)
        scores = torch.tanh(key.matmul(mids).squeeze() + self.bias)   # (batch_size, time_step, 1, 1)
        scores = scores.squeeze()   # (batch_size, time_step)
        scores = scores - scores.max(dim=1, keepdim=True)[0]
        scores = torch.exp(scores) * mask
        attn_weights = scores / scores.sum(dim=1, keepdim=True)
        return attn_weights


class IAN(nn.Module):

    def __init__(self, config):
        super(IAN, self).__init__()
        self.vocab_size = config['vocab_size']
        self.embedding_size = config['embedding_size']
        self.hidden_size = config['hidden_size']
        self.n_class = config['n_class']
        self.l2_reg = config['l2_reg']
        self.max_aspect_len = config['max_aspect_len']
        self.max_context_len = config['max_context_len']

        self.embedding = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.embedding_size)
        self.aspect_lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_size, batch_first=True)
        self.context_lstm = nn.LSTM(input_size=self.embedding_size, hidden_size=self.hidden_size, batch_first=True)
        self.aspect_attn = Attention(self.hidden_size, self.hidden_size)
        self.context_attn = Attention(self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(config['dropout'])
        self.fc = nn.Linear(self.hidden_size * 2, self.n_class)
        self.embedding.weight.data.copy_(torch.from_numpy(config['embedding']))

    def forward(self, aspect, context, aspect_mask, context_mask):
        aspect = self.embedding(aspect)
        aspect = self.dropout(aspect)
        aspect_output, _ = self.aspect_lstm(aspect)
        aspect_output = aspect_output * aspect_mask.unsqueeze(-1)
        aspect_avg = aspect_output.sum(dim=1, keepdim=False) / aspect_mask.sum(dim=1, keepdim=True)
        context_adj=context
        context = self.embedding(context)
        context = self.dropout(context)
        context_output, _ = self.context_lstm(context)
        context_output = context_output * context_mask.unsqueeze(-1)
        context_avg = context_output.sum(dim=1, keepdim=False) / context_mask.sum(dim=1, keepdim=True)
        aspect_attn = self.aspect_attn(context_avg, aspect_output, aspect_mask).unsqueeze(1)
        aspect_features = aspect_attn.matmul(aspect_output).squeeze()
        context_attn = self.context_attn(aspect_avg, context_output, context_mask).unsqueeze(1)
        context_features = context_attn.matmul(context_output).squeeze()
    
        predicted_context=torch.argmax(context_attn,2).reshape(-1)
        predicted_aspect = torch.argmax(aspect_attn,2).reshape(-1)
      
        features = torch.cat([aspect_features, context_features], dim=1)
        features = self.dropout(features)
        output = self.fc(features)
        output = torch.tanh(output)
        return output, aspect_attn, context_attn, predicted_context, predicted_aspect

In [ ]:
config={}

In [ ]:

config['embedding_size'] = 300
config['batch_size'] =128
config['epochs'] = 10
config['hidden_size'] = 300
config['n_class'] = 3
config['pre_processed'] = False
config['learning_rate']  = 0.0001
config['l2_reg'] = 1e-5
config['clip'] = 10
config['dropout']  = 0.5


In [ ]:
def get_data(path_dataset, dataset,config):

  tokens,max_aspect_len, max_context_len= tokenize(path_dataset, "train.txt", "test.txt")
  config['max_aspect_len'] = max_aspect_len
  config['max_context_len']=max_context_len
  config['vocab_size'] = len(tokens)
  aspects, contexts, labels, aspect_lens, context_lens = read_from_token(tokens, max_aspect_len, max_context_len, dataset,path_dataset)
  saved = save_file(dataset, aspects, contexts, labels, aspect_lens, context_lens,path_dataset)
  return tokens, saved, config 



In [ ]:
tokens,train_data,config = get_data("data/restaurant/", "train",config)
tokens,test_data,config = get_data("data/restaurant/", "test",config)


data/restaurant/test.txt
data/restaurant/test.txt


In [ ]:
config['embedding'] = load_word_embeddings("glove.840B.300d.txt", 300,tokens)

There are 1910 words in vocabulary and 36 words out of vocabulary


In [ ]:
train_dataset = IanDataset(train_data)
test_dataset = IanDataset(test_data)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=True, num_workers=2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch.optim as optim
model = IAN(config).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['l2_reg'])

In [ ]:
max_acc = 0
for epoch in range(10):
  train_total_cases = 0
  train_correct_cases = 0
  for data in train_loader:
    
    aspects, contexts, labels, aspect_masks, context_masks = data
    aspects, contexts, labels = aspects.cuda(), contexts.cuda(), labels.cuda()
    aspect_masks, context_masks = aspect_masks.cuda(), context_masks.cuda()
    optimizer.zero_grad()
    outputs,aspect_attn, context_attn, predcon, predasp = model(aspects, contexts, aspect_masks, context_masks)

    _, predicts = outputs.max(dim=1)
    train_total_cases += labels.shape[0]
    train_correct_cases += (predicts == labels).sum().item()
    loss = criterion(outputs, labels)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), config['clip'])
    optimizer.step()


  train_accuracy = train_correct_cases / train_total_cases
  test_total_cases = 0
  test_correct_cases = 0
  for data in test_loader:
    aspects, contexts, labels, aspect_masks, context_masks = data
    aspects, contexts, labels = aspects.cuda(), contexts.cuda(), labels.cuda()
    aspect_masks, context_masks = aspect_masks.cuda(), context_masks.cuda()
    outputs,aspect_attn, context_attn, predcon, predasp= model(aspects, contexts, aspect_masks, context_masks)
    resCon=[]
    resAsp = []
    phrase =[]
    phrases ={}

    if epoch ==9:
      print(contexts.shape)
      print(aspects.shape)
      for i in range (contexts.shape[0]):
        phrase =[]
        for con in contexts[i]:
          phrase.append(get_word(tokens,con))
        
        word = get_word(tokens,contexts[i][predcon[i]])
        phrases[word] = phrase
        resCon.append(get_word(tokens,contexts[i][predcon[i]]))
        resAsp.append(get_word(tokens,aspects[i][predasp[i]]))

      for i in range(len(resCon)):
        print("Aspect : ", resAsp[i])
        print("Context :", resCon[i])
        print("Phrase : " ,phrases[resCon[i]])
      
    _, predicts = outputs.max(dim=1)
    test_total_cases += labels.shape[0]
   
    test_correct_cases += (predicts == labels).sum().item()
  test_accuracy = test_correct_cases / test_total_cases
  print('[epoch %03d] train accuracy: %.4f test accuracy: %.4f' % (epoch, train_accuracy, test_accuracy))
  max_acc = max(max_acc, test_accuracy)

[epoch 000] train accuracy: 0.6968 test accuracy: 0.7241
[epoch 001] train accuracy: 0.7138 test accuracy: 0.7277
[epoch 002] train accuracy: 0.7138 test accuracy: 0.7312
[epoch 003] train accuracy: 0.7160 test accuracy: 0.7473
[epoch 004] train accuracy: 0.7238 test accuracy: 0.7518
[epoch 005] train accuracy: 0.7263 test accuracy: 0.7357
[epoch 006] train accuracy: 0.7249 test accuracy: 0.7438
[epoch 007] train accuracy: 0.7376 test accuracy: 0.7464
[epoch 008] train accuracy: 0.7368 test accuracy: 0.7500
torch.Size([128, 78])
torch.Size([128, 23])
Aspect :  tacos
Context : .
Phrase :  ['the', 'is', 'absolutely', 'adorable', 'and', 'the', 'food', 'is', 'delicious', '.', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa